In [1]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from keras.preprocessing import image
import numpy as np
from skimage.transform import resize

In [2]:
#Read the video, extract frames from it and save them as images
count = 0
#videoFile = "/content/Tom_and_jerry.mp4"
videoFile = "/content/anni005.mpg"

cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [ ]:
#Read the video, extract frames from it and save them as images
count = 0
videoFile = "/content/Tom_and_Jerry_3.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [3]:
#Label a few images for training the model
#data = pd.read_csv('mapping.csv')
 
data = pd.read_csv('/content/mapping_frame_change_revised.csv')
#test = pd.read_csv('testing.csv')

In [69]:
 #train our model:

#Training images, and
#Their corresponding class
#Since there are three classes, we will one hot encode them using the to_categorical() function
X = []
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)
X = np.array(X)

In [ ]:

test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [70]:
from keras.utils import np_utils
train_y = np_utils.to_categorical(data.Class)
print("training data")
print(train_y)
#test_y = np_utils.to_categorical(test.Class)
#print("test data")
#print(test_y)

training data
[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 

We will be using a VGG16 pretrained model which takes an input image of shape (224 X 224 X 3). Since our images are in a different size, we need to reshape all of them. We will use the resize() function of skimage.transform to do this.

In [71]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224,3)).astype(int)
    image.append(a)
X = np.array(image)

All the images have been reshaped to 224 X 224 X 3. But before passing any input to the model, we must preprocess it as per the model’s requirement. Otherwise, the model will not perform well enough. 

In [72]:
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)
print(X.shape)
#test_image = preprocess_input(test_image)
#print(test_image.shape)

(392, 224, 224, 3)


We also need a validation set to check the performance of the model on unseen images. We will make use of the train_test_split() function of the sklearn.model_selection module to randomly divide images into training and validation set.

In [73]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, train_y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(274, 224, 224, 3)
(118, 224, 224, 3)
(274, 2)
(118, 2)


The next step is to build our model. As mentioned, we shall be using the VGG16 pretrained model for this task. Let us first import the required libraries to build the model:

In [74]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

We will now load the VGG16 pretrained model and store it as base_model:

In [75]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

We will make predictions using this model for X_train and X_valid, get the features, and then use those features to retrain the model.

In [76]:
X_train = base_model.predict(X_train)
print(X_train)
X_valid = base_model.predict(X_valid)
print(X_valid)
#test_image = base_model.predict(test_image)
#print(test_image)

[[[[ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.        ]
   ...
   [ 0.          0.          0.         ...  0.         13.692962
     0.        ]
   [ 0.          0.          0.         ...  0.         20.675089
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.        ]]

  [[ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.        ]
   ...
   [ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          0.          0.         ...  0.         29.699348
     0.        ]
   [ 0.          0.          0.         ...  0.          5.087318
     0.        ]]

  [[ 0.          0.          0.   

In order to pass it to our neural network, we have to reshape it to 1-D.

In [77]:
X_train = X_train.reshape(274, 7*7*512)
#X_train = X_train.reshape(208, 224*224*3)
X_valid = X_valid.reshape(118, 7*7*512)
#X_valid = X_valid.reshape(90, 224*224*3)
#test_image = test_image.reshape(186, 7*7*512)
#test_image = test_image.reshape(186, 224*224*3)

We will now preprocess the images and make them zero-centered which helps the model to converge faster.

In [78]:
train = X_train/X_train.max()
X_valid = X_valid/X_train.max()
#test_image = test_image/test_image.max()

Building the model

In [83]:
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
#model.add(InputLayer((224*224*3,)))
model.add(Dense(units=1024, activation='sigmoid'))   # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=512, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=256, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
#model.add(Dense(3, activation='softmax'))  
model.add(Dense(2, activation='sigmoid'))  

In [84]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 1024)              25691136  
_________________________________________________________________
dropout_21 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_29 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_22 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_23 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 2)                

Compiling the model

In [85]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
class_weights = compute_class_weight('balanced',np.unique(data.Class), data.Class)  # computing weights of different classes
print(class_weights)

[ 0.50385604 65.33333333]


In [ ]:
from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]      # model check pointing based on validation loss

Training of model

In [86]:
model.fit(train, y_train, epochs=100, validation_data=(X_valid, y_valid))
#model.fit(train, y_train, epochs=100, batch_size=16, verbose=2)

Epoch 1/100
9/9 [==============================] - 1s 29ms/step - loss: 0.4413 - accuracy: 0.7506 - val_loss: 0.1639 - val_accuracy: 0.9831
Epoch 2/100
9/9 [==============================] - 0s 32ms/step - loss: 0.0712 - accuracy: 0.9908 - val_loss: 0.1887 - val_accuracy: 0.9831
Epoch 3/100
9/9 [==============================] - 0s 14ms/step - loss: 0.0208 - accuracy: 0.9979 - val_loss: 0.1954 - val_accuracy: 0.9831
Epoch 4/100
9/9 [==============================] - 0s 14ms/step - loss: 0.0528 - accuracy: 0.9955 - val_loss: 0.1932 - val_accuracy: 0.9831
Epoch 5/100
9/9 [==============================] - 0s 13ms/step - loss: 0.0310 - accuracy: 0.9973 - val_loss: 0.1888 - val_accuracy: 0.9831
Epoch 6/100
9/9 [==============================] - 0s 13ms/step - loss: 0.0268 - accuracy: 0.9973 - val_loss: 0.1822 - val_accuracy: 0.9831
Epoch 7/100
9/9 [==============================] - 0s 13ms/step - loss: 0.0463 - accuracy: 0.9955 - val_loss: 0.1732 - val_accuracy: 0.9831
Epoch 8/100
9/9 [===

In [ ]:
predictions = model.predict_classes(test_image)
#print(predictions.shape)
#print(predictions)
#print(test_y)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
scores = model.evaluate(test_image, test_y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

6/6 [==============================] - 0s 4ms/step - loss: 2.0285 - accuracy: 0.5376
accuracy: 53.76%
